<a href="https://colab.research.google.com/github/agg-shambhavi/Gans/blob/main/Pix2Pix/Pix2Pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# File Paths


In [1]:
testing_photo = "/content/drive/MyDrive/Computer Vision /PIX2PIX/Data/testing photo.zip"
testing_sketch = "/content/drive/MyDrive/Computer Vision /PIX2PIX/Data/testing sketch.zip"
training_photo = "/content/drive/MyDrive/Computer Vision /PIX2PIX/Data/training photo.zip"
training_sketch = "/content/drive/MyDrive/Computer Vision /PIX2PIX/Data/training sketch.zip"

In [2]:
gen_checkpoint_dir = "/content/drive/MyDrive/Computer Vision /PIX2PIX/Gen-checkpoint/checkpoint.pth.tar"
disc_checkpoint_dir = "/content/drive/MyDrive/Computer Vision /PIX2PIX/Disc-checkpoint/checkpoint.pth.tar"
results = "/content/drive/MyDrive/Computer Vision /PIX2PIX/Result "

In [3]:
  train_photo_dir = "/content/training photo"
  train_sketch_dir = "/content/training sketch"
  test_photo_dir = "/content/testing photo"
  test_sketch_dir = "/content/testing sketch"

# Hyper parameters

In [4]:
# Hyperparameters
LEARNING_RATE = 2e-4
BATCH_SIZE = 16
IMAGE_SIZE = 256
CHANNELS_IMG = 3
L1_LAMBDA = 100
LAMBDA_GP = 10
NUM_EPOCHS = 100

# Imports

In [5]:
!pip install -U git+https://github.com/albu/albumentations > /dev/null 

  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-m7s20atn


In [6]:
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.optim as optim
import zipfile
import numpy as np
from tqdm import tqdm
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Preprocessing

In [8]:
def extract_zip_file(path):
  # extract all contents of zip file
  zip_train = zipfile.ZipFile(path, 'r')
  zip_train.extractall("./")
  zip_train.close()

In [9]:
extract_zip_file(testing_photo)
extract_zip_file(testing_sketch)
extract_zip_file(training_photo)
extract_zip_file(training_sketch)

In [10]:
both_transform = A.Compose(
    [A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
     A.Resize(width=256, height=256),
     ToTensorV2()], additional_targets={"image0": "image"},
)


class SketchDataset(Dataset):
    def __init__(self, root_dir_photos, root_dir_sketch):
        self.root_dir_photos = root_dir_photos
        self.root_dir_sketch = root_dir_sketch
        self.list_photos = os.listdir(self.root_dir_photos)
        self.list_sketch = os.listdir(self.root_dir_sketch)
        self.final_list = []
        for file in self.list_photos:
          name_file = file.split(".")[0].split("-")
          stch_file_name_1 = str(name_file[0]) + "2-" + str(name_file[1]) + "-" + str(name_file[2]) + "-sz1.jpg"
          stch_file_name_2 = str(name_file[0]) + "1-" + str(name_file[1]) + "-" + str(name_file[2]) + "-sz1.jpg"
          stch_file_name_3 = str(name_file[0]) + "-" + str(name_file[1]) + "-" + str(name_file[2]) + "-sz1.jpg"

          if stch_file_name_1 in self.list_sketch:
            self.final_list.append([file, stch_file_name_1])
          elif stch_file_name_2 in self.list_sketch:
            self.final_list.append([file, stch_file_name_2])
          elif stch_file_name_3 in self.list_sketch:
            self.final_list.append([file, stch_file_name_3])



    def __len__(self):
        assert len(self.list_photos) == len(self.list_sketch)
        return len(self.list_photos)

    def __getitem__(self, index):
        photo_file = self.final_list[index][0]
        sketch_file = self.final_list[index][1]
        photo_path = os.path.join(self.root_dir_photos, photo_file)
        sketch_path = os.path.join(self.root_dir_sketch, sketch_file)
        photo = np.array(Image.open(photo_path).convert('RGB'))
        sketch = np.array(Image.open(sketch_path).convert('RGB'))
        augmentations = both_transform(image=sketch, image0=photo)
        sketch = augmentations["image"]
        photo = augmentations["image0"]
        return sketch, photo

# Model Archiecture

In [11]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=4,
                stride=stride,
                bias=False,
                padding_mode="reflect",
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):  # 256 -> 30 x 30
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                # we are getting the input and output image both concatenated channel wise
                in_channels * 2,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                CNNBlock(
                    in_channels, feature, stride=1 if feature == features[-1] else 2
                ),
            )
            in_channels = feature

        layers.append(
            nn.Conv2d(
                in_channels,
                1,
                kernel_size=4,
                stride=1,
                padding=1,
                padding_mode="reflect",
            )
        )
        self.model = nn.Sequential(*layers)

    def forward(self, x, y):
        x = torch.cat([x, y], dim=1)
        x = self.initial(x)
        return self.model(x)

In [12]:
class Block(nn.Module):
    def __init__(
        self, in_channels, out_channels, down=True, act="relu", use_dropout=False
    ):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
                bias=False,
            )
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
        self.down = down

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x


class Generator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()
        self.in_channels = in_channels
        self.initial_down = nn.Sequential(
            nn.Conv2d(self.in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )

        self.down1 = Block(
            features, features * 2, down=True, act="leaky", use_dropout=False
        )
        self.down2 = Block(
            features * 2, features * 4, down=True, act="leaky", use_dropout=False
        )
        self.down3 = Block(
            features * 4, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down4 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down5 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down6 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features * 8, features * 8, 4, 2, 1, padding_mode="reflect"),
            nn.ReLU(),
        )
        self.up1 = Block(
            features * 8, features * 8, down=False, act="relu", use_dropout=True
        )
        self.up2 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True
        )
        self.up3 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True
        )
        self.up4 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=False
        )
        self.up5 = Block(
            features * 8 * 2, features * 4, down=False, act="relu", use_dropout=False
        )
        self.up6 = Block(
            features * 4 * 2, features * 2, down=False, act="relu", use_dropout=False
        )
        self.up7 = Block(
            features * 2 * 2, features, down=False, act="relu", use_dropout=False
        )
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(
                features * 2, in_channels, kernel_size=4, stride=2, padding=1
            ),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], dim=1))
        up3 = self.up3(torch.cat([up2, d6], dim=1))
        up4 = self.up4(torch.cat([up3, d5], dim=1))
        up5 = self.up5(torch.cat([up4, d4], dim=1))
        up6 = self.up6(torch.cat([up5, d3], dim=1))
        up7 = self.up7(torch.cat([up6, d2], dim=1))
        return self.final_up(torch.cat([up7, d1], dim=1))

In [13]:
def save_some_examples(gen, val_loader, epoch, folder):
    x, y = next(iter(val_loader))
    x, y = x.to(device), y.to(device)
    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5 # remove normalization
        save_image(y_fake, folder + f"/{epoch}_fake.png")
        save_image(x * 0.5 + 0.5, folder + f"/{epoch}_input.png")
        save_image(y * 0.5 + 0.5, folder + f"/{epoch}_truth.png")
        # if epoch == 1:
        #     save_image(y*0.5 + 0.5, folder + f"/{epoch}_real.png")
    gen.train()
    
def save_checkpoint(model, optimizer, filename):
    print("=> Saving Checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict()
    }
    torch.save(checkpoint, filename) 

In [14]:
def train_fn(disc, gen, loader, opt_disc, opt_gen, l1, bce):
  loop = tqdm(loader, leave=True)
  for idx, (x,y) in enumerate(loop):
    x, y = x.to(device), y.to(device)

    # Train Discriminator
    y_fake = gen(x)
    D_real = disc(x, y)
    D_fake = disc(x, y_fake.detach())
    D_real_loss = bce(D_real, torch.ones_like(D_real))
    D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
    D_loss = (D_real_loss + D_fake_loss) / 2
    disc.zero_grad()
    D_loss.backward()
    opt_disc.step()

    # Train generator 
    D_fake = disc(x, y_fake)
    G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
    L1 = l1(y_fake, y) * L1_LAMBDA
    G_loss = G_fake_loss + L1
    gen.zero_grad()
    G_loss.backward()
    opt_gen.step()


In [15]:
def main():
  disc = Discriminator(in_channels=3).to(device)
  gen = Generator(in_channels=3).to(device)
  opt_disc = optim.Adam(disc.parameters(), lr= LEARNING_RATE, betas=(0.5, 0.999))
  opt_gen = optim.Adam(gen.parameters(), lr= LEARNING_RATE, betas=(0.5, 0.999))
  BCE = nn.BCEWithLogitsLoss()
  L1_LOSS = nn.L1Loss()

  train_dataset = SketchDataset(root_dir_photos=train_photo_dir, root_dir_sketch=train_sketch_dir)
  test_dataset = SketchDataset(root_dir_photos=test_photo_dir, root_dir_sketch=test_sketch_dir)

  train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  val_loader = DataLoader(test_dataset, batch_size=2, shuffle=True)

  for epoch in range(1, NUM_EPOCHS+1):
    print(f"Epoch {epoch}")
    train_fn(disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE)

    if epoch == NUM_EPOCHS - 1:
      save_checkpoint(gen, opt_gen, filename = gen_checkpoint_dir)
      save_checkpoint(disc, opt_disc, filename = disc_checkpoint_dir )

    save_some_examples(gen, val_loader, epoch, folder=results)

In [16]:
if __name__ == "__main__":
  main()

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 2


  0%|          | 0/6 [00:00<?, ?it/s]

...

100%|██████████| 6/6 [02:57<00:00, 29.53s/it]


=> Saving Checkpoint
=> Saving Checkpoint


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 100


100%|██████████| 6/6 [03:01<00:00, 30.18s/it]
